<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

# Python

## bonnes pratiques de test

# catégories de test

* on trouve souvent dans la littérature des distinctions comme
  * test unitaires
  * test d'intégration
  * tests système, etc...
  * (non-régression)
* peuvent faire du sens au niveau d'un projet

## catégorisation - suite

* mais pas sûr que ces distinctions soient pertinentes/utiles
  * pour comparer deux projets

* exemple
  * un test système pour une librairie d'algèbre linéaire
  * peut être plus simple à mettre en place
  * qu'un test unitaire pour un système de téléphonie

## pourquoi automatiser les tests ?

* nécessité de tester au moins une fois
* un tout petit changement peut tout casser
* il faut donc **tout tester à chaque changement**
* de ce point de vue tous les tests sont de **non-régression**
  * sauf la première fois
  * lorsqu'on teste le test

## catégories de test - pratique

* il faut que les scénarii de test soient reproductibles
  * nécessaire de contrôler entièrement l'environnement
* il reste une catégorisation objective
  * selon la complexité de l'environnement de test

## aujourd'hui

* grâce à des outils comme
  * docker - pour la gestion des environnements
  * et l'intégration continue (webhooks gitlab/github)
  * (présentation séparée)
* il est à la portée d'un *simple individu* de 
  * mettre en place des tests **systématiques** très complets 
  * tant que tous les composants tiennent dans une VM
* et même, au prix d'un effort un peu supérieur
  * d'orchestrer plusieurs VMs
  * sur un cloud comme amazon ou autre

### écrire les tests en même temps que le code

* c'est pourquoi il est entendu que
* on écrit les tests en même temps que le code
  * que ce soit pour du code from scratch
  * ou des corrections de bug
* nécessaire mais généralement pas suffisant
  * intégration/système
* trouver le bon compromis

# librairies de test

* les frameworks de test les plus cités
  * `unittest` - dans la librairie standard
  * `pytest` - à installer avec `pip`
  * (`nose` - à installer avec `pip` - pas présenté)
* la tendance est en faveur de `pytest`
* signalons en outre `doctest` (voir partie sur la doc)
  * beaucoup moins puissant
  * mais avantage de grouper code et test

# `unittest`

fait partie de la librairie standard; contient:

* une interface orientée objet
  * pour l'écriture des tests
* une fonctionnalité *runner*
  * exécution et présentation des résultats
* et une fonctionnalité *discover*
  * recherche de tous les test-cases
  * e.g. dans tout un package ou module

## un exemple

In [1]:
!cat library/pgcd.py

#!/usr/bin/env python3
def pgcd(a, b):
    """
    Le pgcd de a et b par l'algorithme d'Euclide
    >>> pgcd(42, 30)
    6
    >>> pgcd(30, 42)
    6
    """
    if b > a :
        a, b = b, a
    while True:
        r = a % b
        if r == 0:
            return b
        a, b = b, r


In [20]:
!cat tests/test_pgcd_unittest.py

from library.pgcd import pgcd

from unittest import TestCase

class TestPgcd(TestCase):

    def test_upper(self):
        self.assertEqual(pgcd(42, 30), 6)

    def test_lower(self):
        self.assertEqual(pgcd(30, 42), 6)


## à noter

* ces tests sont simplistes (⇔ à ceux en doctest)
  * mais on faire beaucoup plus !
* les vérifications sont faites à partir de **méthodes**
  * de la classe `TestCase` - ici `assertEqual`
* notamment on peut vérifier qu'un appel lève une exception 
  * avec `assertRaises` et similaires
* ou que deux valeurs sont presques égales (précision flottants)
  * avec `AssertAlmostEqual`
* ou qu'un string matche une expression régulière
  * avec `assertRegex`

## comment sont découverts les tests

* la convention de nommer les objets en `test_*`:
  * le module s'appelle `test_pgcd.py`
  * la méthode s'appelle `test_upper` 
  * c'est important pour les fonctions de découverte
* la classe hérite de `TestCase` 
  * c'est pourquoi son nom importe peu
  * (les noms de classes sont `EnChasseMixte`)

## quoi en faire

* point d'entrée `python3 -m unittest`
* on peut lui passer un module, une classe ou une méthode
  * ou une liste de .. évidemment
* ou le laisser trouver tous les tests dans un module
  * fonction `discover`

In [3]:
# si on lui passe une méthode précise, seul ce test case est lancé
!python3 -m unittest tests.test_pgcd_unittest.TestPgcd.test_upper

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [4]:
# si on lui précise le module, les deux tests sont lancés
!python3 -m unittest tests.test_pgcd_unittest

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


In [22]:
# idem avec l'option -v 
!python3 -m unittest -v tests.test_pgcd_unittest

test_lower (tests.test_pgcd_unittest.TestPgcd) ... ok
test_upper (tests.test_pgcd_unittest.TestPgcd) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


In [23]:
# en fait dans le répertoire tests/ 
# il y a plus de testcases que cela
!ls tests

__init__.py		 test_pgcd_pytest.py	     test_pgcd_unittest.py
__pycache__		 test_pgcd_pytest_broken.py  test_pgcd_unittest_fix1.py
test_pgcd_nose_class.py  test_pgcd_pytest_class.py   test_pgcd_unittest_fix2.py
test_pgcd_nose_deco.py	 test_pgcd_pytest_raise.py


In [21]:
# on lance la découverte sur tout le package...
# du coup il va trouver les autres versions du même test
# que nous allons voir tout de suite
!python3 -m unittest discover -v tests

test_lower (test_pgcd_unittest.TestPgcd) ... ok
test_upper (test_pgcd_unittest.TestPgcd) ... ok
test_lower (test_pgcd_unittest_fix1.TestPgcd) ... méthode/fixture - setup - test_pgcd_unittest_fix1.TestPgcd.test_lower
méthode/fixture - tearDown - test_pgcd_unittest_fix1.TestPgcd.test_lower
ok
test_upper (test_pgcd_unittest_fix1.TestPgcd) ... méthode/fixture - setup - test_pgcd_unittest_fix1.TestPgcd.test_upper
méthode/fixture - tearDown - test_pgcd_unittest_fix1.TestPgcd.test_upper
ok
classe/fixture - setup - TestPgcd
test_lower (test_pgcd_unittest_fix2.TestPgcd) ... ok
test_upper (test_pgcd_unittest_fix2.TestPgcd) ... ok
classe/fixture - tearDown - TestPgcd

----------------------------------------------------------------------
Ran 6 tests in 0.001s

OK


## fixtures

* c'est quoi une fixture ?
* le code pour mettre le système dans un état initial connu
  * assez rustique dans `unittest`
* on veut pouvoir définir simplement
  * une façon d'initialiser/nettoyer (setup/teardown)
* à l'entrée et la sortie de **tout le scénario**
  * `setUpClass/tearDownClass`
* et aussi à l'entrée et la sortie de **chaque test**
  * `setUp`/`tearDown`

## `setUp`/`tearDown`

In [7]:
!cat tests/test_pgcd_unittest_fix1.py

from library.pgcd import pgcd

from unittest import TestCase

class TestPgcd(TestCase):

    # en définissant ces deux méthodes
    # on obtient du code qui est exécuté
    # avant et après CHAQUE TEST
    # soit donc ici DEUX FOIS
    def setUp(self):
        print("méthode/fixture - setup - {}".format(self.id()))

    def tearDown(self):
        print("méthode/fixture - tearDown - {}".format(self.id()))
    
    def test_upper(self):
        self.assertEqual(pgcd(42, 30), 6)

    def test_lower(self):
        self.assertEqual(pgcd(30, 42), 6)


In [8]:
!python3 -m unittest tests.test_pgcd_unittest_fix1

méthode/fixture - setup - tests.test_pgcd_unittest_fix1.TestPgcd.test_lower
méthode/fixture - tearDown - tests.test_pgcd_unittest_fix1.TestPgcd.test_lower
.méthode/fixture - setup - tests.test_pgcd_unittest_fix1.TestPgcd.test_upper
méthode/fixture - tearDown - tests.test_pgcd_unittest_fix1.TestPgcd.test_upper
.
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## `setUpClass` et `teardownClass`

In [9]:
!cat tests/test_pgcd_unittest_fix2.py

from library.pgcd import pgcd

from unittest import TestCase

class TestPgcd(TestCase):

    # en définissant ces deux méthodes
    # on obtient du code qui est exécuté
    # avant et après LE PAQUET de tests
    # soit donc UNE SEULE FOIS
    @classmethod
    def setUpClass(cls):
        print("classe/fixture - setup - {}".format(cls.__name__))

    @classmethod
    def tearDownClass(cls):
        print("classe/fixture - tearDown - {}".format(cls.__name__))
    
    def test_upper(self):
        self.assertEqual(pgcd(42, 30), 6)

    def test_lower(self):
        self.assertEqual(pgcd(30, 42), 6)


In [10]:
!python3 -m unittest tests.test_pgcd_unittest_fix2

classe/fixture - setup - TestPgcd
..classe/fixture - tearDown - TestPgcd

----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## `unittest` - autres traits

* on peut aussi définir `setUpModule`/`tearDownModule` 
  * au niveau du module..
* avec les décorateurs `skip` `skipIf` `skipUnless`
  * on peut passer des tests en fonction de l'environnement
  * typiquement de l'operating system
  * ou de la version python ...
  * on peut aussi passer un test pendant le run avec [`skipTest()`](https://docs.python.org/3.5/library/unittest.html#unittest.TestCase.skipTest)
* avec la notion de [`subTest`](https://docs.python.org/3.5/library/unittest.html#unittest.TestCase.subTest)
  * on peut éviter que la première assertion 
  * ne cause la fin de toute la méthode

## `unittest` - épilogue

* bref, c'est très complet
* mais un tout petit peu compliqué
* par exemple
  * le niveau 'classe' peut être jugé superflu
  * dans notre exemple, avec un layout tout bête
  * un test = niveau 4 !

`tests.test_pgcd_unittest.TestPgcd.test_lower`

# `pytest`

## `pytest` - introduction

* philosophie générale
  * "no boilerplate, no required api" 
* supporte *aussi* les tests écrits en `unittest`
* format de sortie le plus lisible
  * notamment pour les tests qui ne passent pas
  * entre autres une raison de son succès
* [la documentation sur readthedocs](http://doc.pytest.org/en/latest/assert.html)
  * système de plugins disponible  

## installation

```
pip3 install pytest
```

* expose une commande `py.test` (⇔ `python3 -m pytest`)

## lancement des tests / discovery 

In [24]:
# tout simplement
!py.test tests

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-4.3.0, py-1.8.0, pluggy-0.8.1
rootdir: /Users/tparment/git/flotpython-slides/slides-tests, inifile:
collected 17 items                                                             

tests/test_pgcd_nose_class.py ..                                         [ 11%]
tests/test_pgcd_nose_deco.py ..                                          [ 23%]
tests/test_pgcd_pytest.py ..                                             [ 35%]
tests/test_pgcd_pytest_broken.py F.                                      [ 47%]
tests/test_pgcd_pytest_class.py ..                                       [ 58%]
tests/test_pgcd_pytest_raise.py .                                        [ 64%]
tests/test_pgcd_unittest.py ..                                           [ 76%]
tests/test_pgcd_unittest_fix1.py ..                                      [ 88%]
tests/test_pgcd_unittest_fix2.py ..                       

In [25]:
# ou seulement sur un module, une classe, un testcase
!py.test -v tests/test_pgcd_pytest.py

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-4.3.0, py-1.8.0, pluggy-0.8.1 -- /Users/tparment/git/flotpython-slides/venv/bin/python3.7
cachedir: .pytest_cache
rootdir: /Users/tparment/git/flotpython-slides/slides-tests, inifile:
collected 2 items                                                              

tests/test_pgcd_pytest.py::test_upper PASSED                             [ 50%]
tests/test_pgcd_pytest.py::test_lower PASSED                             [100%]

=========================== 2 passed in 0.01 seconds ===========================


In [26]:
!py.test -v tests/test_pgcd_pytest.py::test_upper

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-4.3.0, py-1.8.0, pluggy-0.8.1 -- /Users/tparment/git/flotpython-slides/venv/bin/python3.7
cachedir: .pytest_cache
rootdir: /Users/tparment/git/flotpython-slides/slides-tests, inifile:
collected 1 item                                                               

tests/test_pgcd_pytest.py::test_upper PASSED                             [100%]

=========================== 1 passed in 0.01 seconds ===========================


# exemple simpliste

In [14]:
# un test dans sa forme la plus simple
!cat tests/test_pgcd_pytest.py

from library.pgcd import pgcd

def test_upper():
    assert pgcd(42, 30) == 6

def test_lower():
    assert pgcd(30, 42) == 6


In [27]:
!py.test tests/test_pgcd_pytest.py

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-4.3.0, py-1.8.0, pluggy-0.8.1
rootdir: /Users/tparment/git/flotpython-slides/slides-tests, inifile:
collected 2 items                                                              

tests/test_pgcd_pytest.py ..                                             [100%]

=========================== 2 passed in 0.01 seconds ===========================


In [28]:
# et bien sûr toujours le mode bavard
!py.test -v tests/test_pgcd_pytest.py

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-4.3.0, py-1.8.0, pluggy-0.8.1 -- /Users/tparment/git/flotpython-slides/venv/bin/python3.7
cachedir: .pytest_cache
rootdir: /Users/tparment/git/flotpython-slides/slides-tests, inifile:
collected 2 items                                                              

tests/test_pgcd_pytest.py::test_upper PASSED                             [ 50%]
tests/test_pgcd_pytest.py::test_lower PASSED                             [100%]

=========================== 2 passed in 0.01 seconds ===========================


## attendre une exception

In [17]:
# pour spécifier qu'une expression doit retourner une exception
!cat tests/test_pgcd_pytest_raise.py

from library.pgcd import pgcd

import pytest

def test_zero():
    with pytest.raises(ZeroDivisionError):
        pgcd(12, 0)


In [29]:
!py.test tests/test_pgcd_pytest_raise.py

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-4.3.0, py-1.8.0, pluggy-0.8.1
rootdir: /Users/tparment/git/flotpython-slides/slides-tests, inifile:
collected 1 item                                                               

tests/test_pgcd_pytest_raise.py .                                        [100%]

=========================== 1 passed in 0.01 seconds ===========================


## presque égal

In [32]:
## almost-equal
# pas trouvé de méthode native pytest pour cela
# numpy a des outils pour le faire

In [33]:
import numpy as np
x1 = np.array([1e10, 1e-7])
x2 = np.array([1.000001e10, 1e-8])
np.isclose(x1, x2)

array([ True, False])

In [35]:
x3 = 1.00001 * x1
assert all(np.isclose(x1, x3))

In [ ]:
# PS: apparemment cela est maintenant dans pytest 3.0
# voir https://stackoverflow.com/questions/8560131/pytest-assert-almost-equal

## fixtures

* mêmes possibilités que les autres frameworks
* aussi disponibles avec 
  * les tests dans des classes
  * ou directement dans le module

In [36]:
!cat tests/test_pgcd_pytest_class.py

from library.pgcd import pgcd

class TestPgcd:

    def setup(self):
        print("setup")

    def teardown(self):
        print("teardown")

    def setup_class(cls):
        print("\nclass-level setup {}".format(cls.__name__))
        
    def teardown_class(cls):
        print("\nclass-level teardown {}".format(cls.__name__))
        
    def setup_method(self, method):
        print("method-level setup {}".format(method.__name__))
        
    def teardown_method(self, method):
        print("method-level teardown {}".format(method.__name__))
        
    def test_upper(self):
        assert pgcd(42, 30) == 6

    def test_lower(self):
        assert pgcd(30, 42) == 6


In [37]:
# pareil que pour nose, en mettant -s on supprime la capture
!py.test -s tests/test_pgcd_pytest_class.py

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-4.3.0, py-1.8.0, pluggy-0.8.1
rootdir: /Users/tparment/git/flotpython-slides/slides-tests, inifile:
collected 2 items                                                              

tests/test_pgcd_pytest_class.py 
class-level setup TestPgcd
method-level setup test_upper
setup
.teardown
method-level teardown test_upper
method-level setup test_lower
setup
.teardown
method-level teardown test_lower

class-level teardown TestPgcd


=========================== 2 passed in 0.01 seconds ===========================


In [38]:
# un exemple de sortie avec un test qui ne passe pas
!cat tests/test_pgcd_pytest_broken.py

from library.pgcd import pgcd

def test_upper():
    # broken test on purpose
    assert pgcd(42, 30) == 7

def test_lower():
    assert pgcd(30, 42) == 6


In [39]:
# pareil que pour nose, en mettant -s on supprime la capture
!py.test -s tests/test_pgcd_pytest_broken.py

============================= test session starts ==============================
platform darwin -- Python 3.7.0, pytest-4.3.0, py-1.8.0, pluggy-0.8.1
rootdir: /Users/tparment/git/flotpython-slides/slides-tests, inifile:
collected 2 items                                                              

tests/test_pgcd_pytest_broken.py F.

=================================== FAILURES ===================================
__________________________________ test_upper __________________________________

    def test_upper():
        # broken test on purpose
>       assert pgcd(42, 30) == 7
E       assert 6 == 7
E        +  where 6 = pgcd(42, 30)

tests/test_pgcd_pytest_broken.py:5: AssertionError
====================== 1 failed, 1 passed in 0.06 seconds ======================


### fixtures - suite

* toute une ménagerie d'exemples [sur le site pytest](http://doc.pytest.org/en/latest/example/index.html)
  * [commencer par notamment cette page sur les fixtures](http://doc.pytest.org/en/latest/fixture.html#fixtures)
  * qui explique bien ...
* les bases de l'exemple du projet minisim
  * pour une fixture qui définit des variables globales
  * [implémentée ici](https://gitlab.com/parmentelat/minisim2/blob/master/tests/conftest.py)
* notez bien le fichier 'spécial' `conftest.py` qui est chargé automatiquement

# pratiques courantes

* on place généralement les tests 
  * dans un directory `tests/`
  * directement à la racine
  * ou dans le package principal
  * mais ce n'est pas une obligation
* les tests unitaires sont groupés par module source, e.g.
  * `minisim/zone.py`
  * `tests/test_zone.py`
* pour des tests de plus grande portée 
  * il n'y a pas spécialement d'usage 
  * le principal c'est de s'y retouver

# exercice

* s'entraîner a lancer `py.test`
  * sur un test précis
  * sur tout un module de test

* aller voir aussi les pipelines sur gitlab 
  * comment `py.test` est connecté à gitlab-ci
  * où voir les résultats des tests
  
* peut-on améliorer les tests de minisim ?
  * https://gitlab.com/parmentelat/minisim2
  * fixtures ou pas fixtures ...

# Fin

# `nose`

> nose extends unittest to make testing easier

> "no boilerplate, some api"

## installation

* sans surprise:

```
pip3 install nose
```

* expose la commande `nosetests` ($\Longleftrightarrow$ `python3 -m nose`)

In [ ]:
!cat tests/test_pgcd_nose.py

In [ ]:
# lancer tous les tests dans un module
!nosetests tests/test_pgcd_nose.py

In [ ]:
# idem en mode bavard
!nosetests -v tests/test_pgcd_nose.py

* On peut tout aussi bien faire des classes tout de même aussi
  * toujours utile pour les fixtures
  * mais aussi disponible à base de décorateurs
  * gamme complète disponible
* **ATTENTION** par défaut les outputs sont capturés
  * utiliser `-s` pour éviter la capture
* et toujours `-v`/`--verbose` pour le mode bavard

In [ ]:
# des fixtures en utilisant une classe
!cat tests/test_pgcd_nose_class.py

In [ ]:
# avec une classe
!nosetests -v -s tests/test_pgcd_nose_class.py

In [ ]:
# des fixtures en utilisant un decorateur
!cat tests/test_pgcd_nose_deco.py

In [ ]:
# on peut désigner un nom de fichier ou un module python
!nosetests -v -s tests.test_pgcd_nose_deco

## langage d'assertions

### sous-tests

```
def test_evens():
    for i in range(0, 5):
        yield check_even, i, i*3

def check_even(n, nn):
    assert n % 2 == 0 or nn % 2 == 0
```

## discovery

In [ ]:
# l'interface est la plus simple possible
!nosetests -s tests

## à noter

* `nosetests` a trouvé les tests que nous avons écrit pour `nose`
  * **et** ceux écrits en unittest !
* dans l'environnement du cours j'ai beaucoup de bazar
  * j'ai dû préciser `python3 -m unittest tests` 
  * sans préciser `testing` j'obtenais un gros crash
  * alors que `nosetests` tout court fonctionne correctement
* voir aussi [la doc complète sur readthedocs](http://nose.readthedocs.io/en/latest/testing.html)
* on peut exécuter les tests `doctest` depuis `nose`
* un système de plugins